https://tykimos.github.io/2017/08/07/Keras_Install_on_Windows/
    
    # 김태영님 캐글콜리아.
    => 프로젝트 디렉토리 만듦, 가상환경 만들어서 가상환경 실행하기
    # 수업때 진행했던 것처럼. 아나콘다 jupyter 가상환경 venv 만들었고
    # 여기다가  주요패키지 설치할것
    # 딥러닝 라이브러리 설치 => 텐서플로우 2.1 버전 업그레이드
    
    # 케라스 설치 2.3.1
    
 1.14버전으로 실행중


In [0]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15806710144497765256, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5424978570343097419
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "/content/drive/My Drive/untitled1"
categories = ["beagle", "corgi","dach","husky" , "golden" ] # 각 카테고리 폴더에서 불러오기
nb_classes = len(categories)

image_w = 100
image_h = 100

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./multi_image_data.npy", xy)

print("ok", len(y))

beagle  파일 길이 :  803
beagle  :  /content/drive/My Drive/untitled1/beagle/beagle (1).jpg
beagle  :  /content/drive/My Drive/untitled1/beagle/beagle645.jpg
corgi  파일 길이 :  970
corgi  :  /content/drive/My Drive/untitled1/corgi/corgi 102.jpg
corgi  :  /content/drive/My Drive/untitled1/corgi/corgi2916.jpg
dach  파일 길이 :  945
dach  :  /content/drive/My Drive/untitled1/dach/dachs109.jpg
dach  :  /content/drive/My Drive/untitled1/dach/docss (69).jpg
husky  파일 길이 :  502
husky  :  /content/drive/My Drive/untitled1/husky/husky1.jpg
golden  파일 길이 :  806
golden  :  /content/drive/My Drive/untitled1/golden/gold (1).jpg
golden  :  /content/drive/My Drive/untitled1/golden/gold_dog429.jpg
ok 4026


In [0]:
import os, glob, numpy as np
import tensorflow.keras as keras
# module 'tensorflow' has no attribute 'get_default_graph'

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy


import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf

config = tf.ConfigProto() 
''' 텐서플로우 2. 에서 구동안됨
module 'tensorflow' has no attribute 'ConfigProto'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
'''
session = tf.Session(config=config) #자동세션기능
X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


categories = ["corgi", "golden" ,"dach" ,"husky" ,"beagle"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255


(3019, 100, 100, 3)
3019
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 94922029080292902
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1051383428921077374
physical_device_desc: "device: XLA_CPU device"
]


# 모델 평가
앞으로 네트워크를 만들 때 배치 정규화는 웬만하면 적용해 주는 것이 좋다
(논문에 따르면 뒤에서 나올 regularization의 효과도 있어 배치 정규화를 적용하면 
 드랍아웃을 적용하지 않아도 된다고 한다).

https://buomsoo-kim.github.io/keras/2018/04/24/Easy-deep-learning-with-Keras-5.md/

In [0]:
# 배치정규화 시켜서 CNN 모델을 만들자

def cnn_model(): # create model 
    model = Sequential() 
    
    model.add(Conv2D(16, 3, 3, border_mode='same', activation='relu',
                            input_shape=X_train.shape[1:]))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    
    model.add(Flatten()) 
    model.add(Dense(64, activation='relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(64, activation='relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(5, activation='softmax')) 
    return model

model2= cnn_model()
model2.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(100, 100,..., padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 50, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 32)       

In [0]:
def compile_and_train(model, num_epochs):
    model.compile(loss = categorical_crossentropy, optimizer=Adam(), metrics=['acc'])
    history= model.fit(x=X_train, y=y_train, batch_size=32, epochs=num_epochs, verbose=1,
                       validation_split =0.2) 

    
_ = compile_and_train(model2, num_epochs=20)

In [0]:
print("정확도 : %.4f" % (model2.evaluate(X_test, y_test)[1]))

def evaluate_error(model):
    pred = model.predict(X_test, batch_size=32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test))/ y_test.shape[0]
    return error


evaluate_error(model2)

1005/1005 [==============================] - 5s 5ms/step
정확도 : 0.8677


3.743283582089552